# Model Forecasting

## Requirements

In [16]:
#import necesary libraries
import pandas as pd
import os
import pickle
import gc

import matplotlib.pyplot as plt 

from matplotlib.backends.backend_pdf import PdfPages


from utils import load, slicer

## Data Load

In [17]:
df, resample_df = load()

## ARIMA Update & Forecasting

In [18]:
#accesing .pkl files
route = '../stock_forecasting/AutoARIMA_Models/PKL/'
directory = os.fsencode(route)
pdf = PdfPages('../stock_forecasting/AutoARIMA_Models/Forecast/viz.pdf')

window = 15 #number of days to forecast
cont = 0

for file in os.listdir(directory): #for cycle to iterate over folder
   filename = os.fsdecode(file) #get file name
   if filename.endswith(".pkl"): #if ends with .pkl
      cont+=1
      print('Forecasting for:',filename,'(',cont,'/ 31 )') #FLAG

      try: #Splitting
         x = filename.split("_")
         x = x[0]
         df_train, df_val, df_test = slicer(resample_df[(resample_df['Name']==x)])
         
         with open((route+filename).replace(" ",""), 'rb') as f: #load model
            model = pickle.load(f)
      except:
         print('Error in model import: ',x)

      try: #Updating model with new data
         model.update(df_val['Close'])# LOAD DF WITH NEW DATES
         model.update(df_test['Close']) 
         
      except:
         print('Error in model update:',x)
         pass

      try: #forecasting
         forecast = model.predict(n_periods=window,return_conf_int=True)
         forecast_range = pd.date_range(start=df_test.index[-1] + pd.Timedelta('1 day'),periods=window,freq='D')
         forecast_df = pd.DataFrame(forecast[0],index=forecast_range,columns=['Prediction'])
      except:
         print('Error in model forecasting:',x)
         pass
      print('')

      try: #plotting
         pastDF = resample_df[(resample_df['Name']==x)]['Close'].asfreq('D').to_frame()

         fig, ax = plt.subplots(figsize=(15, 8))
         pastDF.dropna(inplace=True)
         pastDF.tail(90).plot(ax=ax, label='Close', linewidth=1)
         forecast_df.plot(ax=ax, label='Forecast', linewidth=1)
         ax.set_title(x+ ' - Close')
         ax.legend();


         pdf.savefig(fig, bbox_inches='tight')
         plt.close(fig) #optimize ram usage
         #fig.data = [] #optimize ram usage
         
      except:
         print('Error in forecast plotting:',x)
         pass

   gc.collect()

pdf.close()

Forecasting for: MMM_autoARIMA_Model.pkl ( 1 / 31 )

Forecasting for: AABA_autoARIMA_Model.pkl ( 2 / 31 )

Forecasting for: DIS_autoARIMA_Model.pkl ( 3 / 31 )

Forecasting for: UTX_autoARIMA_Model.pkl ( 4 / 31 )

Forecasting for: XOM_autoARIMA_Model.pkl ( 5 / 31 )

Forecasting for: AMZN_autoARIMA_Model.pkl ( 6 / 31 )

Forecasting for: TRV_autoARIMA_Model.pkl ( 7 / 31 )

Forecasting for: GS_autoARIMA_Model.pkl ( 8 / 31 )

Forecasting for: AXP_autoARIMA_Model.pkl ( 9 / 31 )

Forecasting for: IBM_autoARIMA_Model.pkl ( 10 / 31 )

Forecasting for: UNH_autoARIMA_Model.pkl ( 11 / 31 )

Forecasting for: WMT_autoARIMA_Model.pkl ( 12 / 31 )

Forecasting for: PFE_autoARIMA_Model.pkl ( 13 / 31 )

Forecasting for: NKE_autoARIMA_Model.pkl ( 14 / 31 )

Forecasting for: CSCO_autoARIMA_Model.pkl ( 15 / 31 )

Forecasting for: CAT_autoARIMA_Model.pkl ( 16 / 31 )

Forecasting for: INTC_autoARIMA_Model.pkl ( 17 / 31 )

Forecasting for: BA_autoARIMA_Model.pkl ( 18 / 31 )

Forecasting for: JPM_autoARIMA_Mode